In [1]:
import pandas as pd
import numpy as np


books = pd.read_csv(r"data/books_data.csv")
reviews = pd.read_csv(r"data/Books_rating.csv")


In [4]:
import re
import os

In [2]:
books = books.head(55000)
reviews = reviews.head(15000)


In [21]:
books.head(2)

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN


In [27]:
year_regex = re.compile('[^\d]*(\d{4})[^\d]*')
def extract_year(x):
    if x:
        result = year_regex.search(x)
        return int(result.group(1)) if result else None
    return None
data = books.copy()
data_sanitized = data.dropna(subset=['Title', 'authors', 'categories', 'publishedDate'])
data_sanitized = data_sanitized[['Title', 'authors', 'publisher', 'categories', 'publishedDate']].copy()
data_sanitized['publishedYear'] = data_sanitized['publishedDate'].apply(extract_year)
data_sanitized['publishedDecade'] = data_sanitized['publishedYear'] // 10 * 10


,Title,authors,publisher,categories,publishedDate,publishedYear,publishedDecade
948,Punktchen Und Anton,['Erich Kästner'],Atrium Verlag AG Zürich,['Juvenile Fiction'],2018-10-01,2018.0,2010.0


In [15]:
from textblob import TextBlob


In [35]:
# Título de libros de la categoría "Ficción" cuyo sentimiento de reseña promedio sea esté en el percentil 90 más alto

def analizar_sentimiento(texto):
    # TextBlob devuelve valores de polaridad de sentimiento en el rango de -1 a 1
    if isinstance(texto, str):
        blob = TextBlob(texto)
        sentimiento = blob.sentiment.polarity
        return sentimiento
    else:
        return None  # Si el texto no es una cadena válida, devolver None

# Sanitizamos el dataset para quedarnos con registros con categorías válidas
#obsolete:data_category_sanitized = data.dropna(subset=['categories'])


# Approach para quedarnos con registros relacionados a fiction
data_category_fiction = data_sanitized[data_sanitized['categories'].str.contains('fiction')]

# Generamos una versión reducida de reseñas que nos sirva para joinear
rating = reviews.copy()
rating_reduced = rating[['Title', 'review/text']]

# Joineamos los libros de la categoría de Ficción con sus reseñas
data_category_fiction_merged_with_ratings = data_category_fiction.merge(rating_reduced, on='Title', how='inner')

# Calculamos el sentimiento sobre los registros de reseñas
data_category_fiction_merged_with_ratings['sentimentPolarity'] = data_category_fiction_merged_with_ratings['review/text'].apply(analizar_sentimiento)

# Agrupamos por título y calculamos el promedio de la sentimientPolarity

data_category_gruped_mean_sentiment_polarity = data_category_fiction_merged_with_ratings.groupby('Title')['sentimentPolarity'].mean().reset_index()

# Ordenamos los registros por polaridad de sentimiento en orden descendente
#obsolete: data_category_fiction_sorted = data_category_gruped_mean_sentiment_polarity.sort_values(by='sentimentPolarity', ascending=False)

# Calculamos el percentil 90
percentile_90 = data_category_gruped_mean_sentiment_polarity['sentimentPolarity'].quantile(0.9)

# Nos quedamos con los libros que esten en el percentil 90
data_category_fiction_percentile_90 = data_category_gruped_mean_sentiment_polarity[data_category_gruped_mean_sentiment_polarity['sentimentPolarity'] >= percentile_90]

data_category_fiction_percentile_90



,Title,sentimentPolarity
6,Gold and greenstone,0.428788
7,Guide to Owning a Birman Cat (Popular Cat Libr...,0.572484
18,Natural Wonders of the World: Converting Measu...,0.425000
21,Sharks/Other Scary Predators (Giant Book of),0.459375


In [33]:
data[data["Title"] == "Punktchen Und Anton"]


,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
948,Punktchen Und Anton,Die Geschichte einer außergewöhnlichen Freunds...,['Erich Kästner'],http://books.google.com/books/content?id=HvKkD...,http://books.google.nl/books?id=HvKkDwAAQBAJ&p...,Atrium Verlag AG Zürich,2018-10-01,https://play.google.com/store/books/details?id...,['Juvenile Fiction'],NaN
